In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
import sqlite3
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
import summary_data

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS

In [2]:
# Join parcel lookup data to parcel files to do some county and district summaries
# Load parcel geography lookup from soundcast db
conn = sqlite3.connect(r'../../../../inputs/db/soundcast_inputs_2023.db')
df_geog_lookup = pd.read_sql_query("SELECT ParcelID, CountyName, district_name, GrowthCenterName FROM parcel_2023_geography", conn)

# Land Use Summary
df_parcels = summary_data.load_landuse('landuse/parcels_urbansim.txt')
# Merge lookup data to parcels
df_parcels = df_parcels.merge(df_geog_lookup, left_on='parcelid', right_on='ParcelID', how='left')

# total parking cost
df_parcels['total_parking_cost'] = df_parcels['pprichrp'] * df_parcels['parkhr_p']

## Parking

### Hourly Parking Cost

- total hourly parking cost in parcel = `parkhr_p` * `pprichrp`

In [3]:
# Daily parking cost totals (ppricdyp)
pd.pivot_table(df_parcels, index='source', values='total_parking_cost', aggfunc='sum').style.format('{:,.0f}')

,total_parking_cost
source,
current run,"54,152,800"


In [4]:
# Parking costs by district
pd.pivot_table(df_parcels, index='district_name', columns='source', values='total_parking_cost', aggfunc='sum').style.format('{:,.0f}')

source,current run
district_name,
East Side,"10,600,400"
Everett-Lynwood-Edmonds,"91,200"
Kitsap,"968,900"
North Seattle-Shoreline,"4,681,000"
Renton-FedWay-Kent,0
S.Kitsap,0
Seattle CBD,"35,261,800"
South Pierce,0
Suburban Snohomish,0


In [5]:
pd.pivot_table(df_parcels, index='GrowthCenterName', columns='source', values='total_parking_cost', aggfunc='sum').style.format('{:,.0f}')

source,current run
GrowthCenterName,
Auburn,0
Bellevue,"10,071,200"
Bothell Canyon Park,0
Bremerton,"548,900"
Burien,0
Everett,"91,200"
Federal Way,0
Greater Downtown Kirkland,0
Issaquah,0


### Parking Supply

- The number of paid public off-street parking spaces with per-hour pricing

In [6]:
# Daily parking cost totals (ppricdyp)
pd.pivot_table(df_parcels, index='source', values='parkhr_p', aggfunc='sum').style.format('{:,.0f}')

,parkhr_p
source,
current run,"202,147"


In [7]:
# Parking costs by district
pd.pivot_table(df_parcels, index='district_name', columns='source', values='parkhr_p', aggfunc='sum').style.format('{:,.0f}')

source,current run
district_name,
East Side,"43,787"
Everett-Lynwood-Edmonds,"13,833"
Kitsap,"17,733"
North Seattle-Shoreline,"18,815"
Renton-FedWay-Kent,37
S.Kitsap,0
Seattle CBD,"80,205"
South Pierce,0
Suburban Snohomish,0


In [8]:
pd.pivot_table(df_parcels, index='GrowthCenterName', columns='source', values='parkhr_p', aggfunc='sum').style.format('{:,.0f}')

source,current run
GrowthCenterName,
Auburn,0
Bellevue,"40,039"
Bothell Canyon Park,0
Bremerton,"9,686"
Burien,0
Everett,"13,641"
Federal Way,0
Greater Downtown Kirkland,0
Issaquah,0


In [9]:
# # Check buffered parcels distance to light rail
# df_buffered = summary_data.load_landuse('landuse/buffered_parcels.txt')

# # Merge geography to file
# df_buffered = df_buffered.merge(df_geog_lookup, left_on='parcelid', right_on='ParcelID', how='left')

In [10]:

# df_buffered.groupby(['source','district_name'])['dist_lbus'].describe()
# # df_buffered.columns